# PCNA Glimpse generation

This notebook is designed to generate large amount of cropped images of cells with the PCNA-irfp marker for a cluster analysis

In [ ]:
import btrack
import matplotlib.pyplot as plt
import os
from skimage.io import imread, imsave, imshow
from tqdm import tqdm
import numpy as np
from pathlib import Path
from natsort import natsorted
import warnings
warnings.filterwarnings('ignore') ###bc imsave throws up low contrast warnings

In [ ]:
expt_root_folder = root_folder

channels = ['003'] ### bf 000 gfp 001 rfp 002 irfp 003

focal_time_range = (0, 800) ### this particular expt stays in focus in this time range

for pos in pos_list:
    print('starting', pos)
    tracks_path = os.path.join(expt_root_folder,'{}/{}/gfp_tracks.h5'.format(expt, pos))

    with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_1") as hdf:
        wt_tracks = hdf.tracks

    ### filter tracks that should contain a full cell cycle in (1 timepoint/frame = 4min, cc time approx 200 frames based on quick look)
    long_tracks = [track for track in wt_tracks if len(track) > 200]

    ### filter tracks that spend a lot of time out of focus
    tracks = [track for track in long_tracks if track.t[-1] < focal_time_range[1]]

    ### show number of filtered tracks
    print('number of glimpses to be generated:', len(tracks))

    ### find images and set output
    image_folder = os.path.join(expt_root_folder,expt, pos, pos+'_stacks')
    output_dir = '/home/nathan/data/kraken/pcna/glimpses/{}/{}'.format(expt, pos)

    for channel in channels:
        ### load images
        print('Loading image channel', channel)
        images = imread(os.path.join(image_folder, 'channel{}.tif'.format(channel)))
        print('Image loaded')
        ### create output directory
        output_ch_dir = os.path.join(output_dir, 'channel'+channel)
        Path(output_ch_dir).mkdir(parents=True, exist_ok=True)
        #os.mkdir(output_ch_dir) if os.path.exists(output_ch_dir) == False else print(output_ch_dir, 'already exists')

        for cell in tqdm(tracks):
            try:
                ### get length of track/glimpse 
                length = range(cell.t[0], cell.t[-1])
                ### get the xy coords of track/glimpse 
                y, x = cell.x, cell.y #### WHY IS THE COORDINATES TRANSPOSED???? 

                ### iterate over all frames cropping
                for j, i in enumerate(length): ## need to have something that solves skipped frames # not including the last frame bc list index out of range ie there is one more i than there
                    ### obtain correct frame
                    image = images[i]
                    ### obtain coordinates of bounding box of glimpse (centered on cell)
                    x1, x2, y1, y2 = x[j], x[j]+200, y[j], y[j]+200 #coordinates to crop by, shifted bearing in mind a further coord shift (+100,+100) occurs due to image padding            
                    ### pad image incase cell goes off FOV
                    image = np.pad(image, 100, 'constant', constant_values = 0) 
                    ## glimpse creation by cropping original image
                    glimpse = image[int(x1): int(x2), int(y1): int(y2)]
                    ### fraction of track this frame is at
                    age = round(i/cell.t[-1],3) 
                    fn = 'cell_ID_' + str(cell.ID) + '_channel' + channel + '_t{}_age{}.tif'.format(i, age)
                    ### glimpse output
                    imsave(os.path.join(output_ch_dir, fn), glimpse) 
                print('cell ID',cell.ID, 'glimpse saved')

            except:
                print(cell.ID, 'failed to produce glimpse')
